In [ ]:
from utils import *

import pickle
import datetime

from scipy.optimize import minimize, root

import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec

In [ ]:
purples = ['#f7fcfd','#e0ecf4','#bfd3e6','#9ebcda','#8c96c6','#8c6bb1','#88419d','#810f7c','#4d004b']
oranges = ['#ffffe5','#fff7bc','#fee391','#fec44f','#fe9929','#ec7014','#cc4c02','#993404','#662506']

# Figure 2

In [ ]:
from utils import *
import pickle
import datetime
import scipy
from scipy.optimize import minimize, root

In [ ]:
def power(Vs,params):
    vg,vb = Vs
    system = anderson(vg,vb,params)
    system.solve()
    return vb*np.sum(system.current[[0,2]])

In [ ]:
def load_voltage(vb,R,system):
    system.change(mulst={0:vb/2, 1:-vb/2, 2:vb/2, 3:-vb/2})
    system.solve()
    return np.sum(system.current[[0,2]])-vb/R

In [ ]:
def main(Vg, Vb, params):
    with ProcessPoolExecutor() as executor:
        results = np.array(list(tqdm(executor.map(solve_bias_gate,Vg,Vb,params),total=len(Vg))),dtype=object)
    return results

In [ ]:
def main_p(Vg, Vb, params):
    with ProcessPoolExecutor() as executor:
        results = np.array(list(executor.map(solve_bias_gate_p,Vg,Vb,params)),dtype=object)
    return results

In [ ]:
U=100
gamma=.25
T=1
dT=.3

In [ ]:
params=Parameters(B=0., u=U, Tl=T+dT, Tr=T, gamma=gamma, kerntype='RTD', dband=1e4, countingleads=[0,2])
params_p=Parameters(B=0., u=U, Tl=T+dT, Tr=T, gamma=gamma, kerntype='Pauli', dband=1e4, countingleads=[0,2])

In [ ]:
system = anderson(0,0,params)
system_p = anderson(0,0,params_p)

In [ ]:
# find max power point
v_max = minimize(power, [-1,-1], args=(params,))
v_max_p = minimize(power, [-1,-1], args=(params_p,))

In [ ]:
# corresponding load voltage and current give load resistance
# RTD
system = anderson(v_max.x[0],v_max.x[1],params)
system.solve()
I_max= system.current[0]+system.current[2]
R_load = v_max.x[1]/I_max
# Pauli
system_p = anderson(v_max_p.x[0],v_max_p.x[1],params_p)
system_p.solve()
I_max_p= system_p.current[0]+system_p.current[2]
R_load_p = v_max_p.x[1]/I_max_p

In [ ]:
R_load, R_load_p

In [ ]:
Vg=np.linspace(-15,15,200)
Vb=[]
for i,vg in tqdm(enumerate(Vg),desc='Finding Vb',total=len(Vg)):
    system = anderson(vg,0,params)
    Vb.append(root(lambda vb: load_voltage(vb,R_load,system),0))

In [ ]:
Vb_p=[]
for i,vg in tqdm(enumerate(Vg),desc='Finding Vb',total=len(Vg)):
    system = anderson(vg,0,params_p)
    Vb_p.append(root(lambda vb: load_voltage(vb,R_load,system),0))

In [ ]:
Vb_load=np.array([vb.x for vb in Vb]).reshape(len(Vg),)
Vb_succes=np.array([vb.success for vb in Vb])
Vb_load_p=np.array([vb.x for vb in Vb_p]).reshape(len(Vg),)
Vb_succes_p=np.array([vb.success for vb in Vb_p])

In [ ]:
plt.plot(Vg,Vb_load)
plt.plot(Vg,Vb_load_p)

In [ ]:
params=Parameters(B=0., u=U, Tl=T+dT, Tr=T, gamma=gamma, kerntype='RTDnoise', dband=1e4, countingleads=[0,2])
res=main(Vg,Vb_load,[params]*len(Vg))


In [ ]:
params_p=Parameters(B=0., u=U, Tl=T+dT, Tr=T, gamma=gamma, kerntype='Pauli', dband=1e4, countingleads=[0,2])
res_p=main_p(Vg,Vb_load_p,[params_p]*len(Vg))


In [ ]:
I=np.array([r[0][0] for r in res])
S=np.array([r[0][1] for r in res])
Q=np.array([np.sum(r[2][[0,2]]) for r in res])
I_o4trunc=np.array([r[3][1] for r in res])
S_o4trunc=np.array([r[3][3] for r in res])
I_std=np.array([np.sum(r[1][[0,2]]) for r in res])

P=-Vb_load*I
P_o4trunc=-Vb_load*I_o4trunc

n = P/Q
nc= 1 - T/(T+dT)

TUR = -S/I*Vb_load/T*(nc-n)/n
TUR_o4trunc = -S_o4trunc/I_o4trunc*Vb_load/T*(nc-n)/n

In [ ]:
I_p=np.array([r[0][0] for r in res_p])
S_p=np.array([r[0][1] for r in res_p])
Q_p=np.array([np.sum(r[2][[0,2]]) for r in res_p])
I_std_p=np.array([np.sum(r[1][[0,2]]) for r in res_p])

P_p=-Vb_load_p*I_p

n_pauli = P_p/Q_p
nc_pauli= 1 - T/(T+dT)

TUR_pauli = -S_p/I_p*Vb_load_p/T*(nc_pauli-n_pauli)/n_pauli



In [ ]:
TUR=-S/I*Vb_load/1*(nc-n)/n
TUR[P<0]=np.nan
TURS=-2*I/Vb_load*1/(nc-n)*n
TURS[P<0]=np.nan
TURI=-S/2*Vb_load/1*(nc-n)/n
TURI[P<0]=np.nan
TURn=1/((-2*I*T)/S/Vb_load+1)
TURn[P<0]=np.nan

TUR_p=-S_p/I_p*Vb_load_p/T*(nc-n_pauli)/n_pauli
TUR_p[P_p<0]=np.nan
TURS_p=-2*I_p/Vb_load_p*T/(nc_pauli-n_pauli)*n_pauli
TURS_p[P_p<0]=np.nan
TURI_p=-S_p/2*Vb_load_p/T*(nc_pauli-n_pauli)/n_pauli
TURI_p[P_p<0]=np.nan
TURn_p=1/((-2*I_p*T)/S_p/Vb_load_p+1)
TURn_p[P_p<0]=np.nan

In [ ]:
# separate TURI in 2 arrays where one contains the positive entries and the other the negative entries
TURI_pos=np.copy(TURI)
TURI_neg=np.copy(TURI)
TURI_pos[TURI<0]=np.nan
TURI_neg[TURI>0]=np.nan
# same for TURI_p
TURI_pos_p=np.copy(TURI_p)
TURI_neg_p=np.copy(TURI_p)
TURI_pos_p[TURI_p<0]=np.nan
TURI_neg_p[TURI_p>0]=np.nan


In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
# plot current, noise, power and  heat current in a 2x2 subplot
fig,ax=plt.subplots(2,3,figsize=(12,6),sharex=True,constrained_layout=True)
curr = ax[0,0].plot(Vg,I/gamma,color=oranges[5])
curr_p = ax[0,0].plot(Vg,I_p/gamma,color=purples[5])
ax[0,0].plot([],[],color='black',linestyle=':')
#ax[0,0].plot(Vg,I_o4trunc)
# plot TURI only where it is positive
#ax[0,0].plot(Vg,TURI_pos,linestyle='--',color=curr[0].get_color())
# shade area above TURI
#ax[0,0].fill_between(Vg,0.007,TURI_pos,color='lightgray', alpha=0.20)
# plot TURI only where it is negative
#ax[0,0].plot(Vg,TURI_neg,linestyle='--',color=curr[0].get_color())
# shade area between TURI and bottom of plot
#ax[0,0].fill_between(Vg,-0.006,TURI_neg,color='lightgray', alpha=0.20)
# same for TURI_p
# ax[0,0].plot(Vg,TURI_p,linestyle='--',color=curr_p[0].get_color())
# ax[0,0].plot(Vg,TURI_pos_p,linestyle='--',color=curr_p[0].get_color())
# ax[0,0].fill_between(Vg,0.007,TURI_pos_p,color='lightgray', alpha=0.20)
# ax[0,0].plot(Vg,TURI_neg_p,linestyle='--',color=curr_p[0].get_color())
# ax[0,0].fill_between(Vg,-0.006,TURI_neg_p,color='lightgray', alpha=0.20)
#ax[0,0].set_ylim(-0.006,0.007)

noise = ax[0,1].plot(Vg,S/gamma,color=curr[0].get_color())
noise_p = ax[0,1].plot(Vg,S_p/gamma,color=curr_p[0].get_color())
#ax[0,1].plot(Vg,S_o4trunc)
#ax[0,1].plot(Vg,TURS,linestyle='--',color=noise[0].get_color())
#ax[0,1].fill_between(Vg,0,TURS,facecolor=noise[0].get_color(),alpha=0.15)
#ax[0,1].plot(Vg,TURS_p,linestyle='--',color=noise_p[0].get_color())
#ax[0,1].fill_between(Vg,0,TURS_p,facecolor=noise_p[0].get_color(),alpha=0.15)
#ax[0,1].set_ylim(0,0.09)


ax[0,2].plot(Vg,np.abs(S/I),color=curr[0].get_color())
ax[0,2].plot(Vg,np.abs(S_p/I_p),color=curr_p[0].get_color())
#ax[0,2].plot(Vg,np.abs(S_o4trunc/I_o4trunc))
ax[0,2].set_ylim(0,10)

ax[1,0].plot(Vg,P/gamma/T,color=curr[0].get_color())
ax[1,0].plot(Vg,P_p/gamma/T,color=curr_p[0].get_color())
#ax[1,0].set_ylim(0,0.00205)

ax[1,1].plot(Vg,Q/gamma,color=curr[0].get_color())
ax[1,1].plot(Vg,Q_p/gamma,color=curr_p[0].get_color())
#ax[1,1].set_ylim(0,0.019)

eff = ax[1,2].plot(Vg,n/nc,color=curr[0].get_color())
eff_p = ax[1,2].plot(Vg,n_pauli/nc_pauli,color=curr_p[0].get_color())
turn, =ax[1,2].plot(Vg,TURn,linestyle=':',color=eff[0].get_color(),label='TUR bound')
ax[1,2].fill_between(Vg,1,TURn,color='lightgray', alpha=0.20)
#ax[1,2].plot(Vg,TURn_p,linestyle=':',color=purples[4])
#ax[1,2].fill_between(Vg,1,TURn_p,color='lightgray', alpha=0.20)
ax[1,2].set_ylim(0,1)
ax[1,2].legend(handles=[turn], bbox_to_anchor=(0.47, 0.75))


# ax[3,0].plot(Vg,TUR)
# ax[3,0].plot(Vg,TUR_pauli)
# # horizontal line at 2
# ax[3,0].axhline(2,linestyle='--',color='black')
# ax[3,0].set_title('TUR')
# ax[3,0].set_ylim(0,5)
# ax[3,1].plot(Vg,Vb_load)
# ax[3,1].plot(Vg,Vb_load_p)
# ax[3,1].set_title('Load voltage')
# # add one legend in the last free spot for all plots

for a in ax.flatten():
    a.set_xmargin(0)

#plt.figlegend(['RTD','Pauli'],loc='upper center',ncol=2)
plt.figlegend(['Second order','First order'],loc='upper center',ncol=2)#, bbox_to_anchor=(0.5, 0.93), ncol=3)


# Loop through all axes to set ticks inside, adjust tick parameters and set titles
paddings = [-35,-30,-30,0,0,0]
titles = [r'Current $I/\Gamma$',r'Noise $S/\Gamma$',r'Fano factor $F=S/I$  ',r'Power $P/(\Gamma \cdot T)$',r'Heat current $Q/\Gamma$',r'Efficiency $n/n_c$']
for i,a in enumerate(ax.flatten()):
    # set ticks
    #a.tick_params(direction='in', top=True, pad=paddings[i])
    # set titles
    a.text(0.98,0.98,titles[i],fontsize='large', horizontalalignment='right',verticalalignment='top',transform=a.transAxes)
    a.text(0.02,0.98,f'({chr(97+i)})',fontsize='large', horizontalalignment='left',verticalalignment='top',transform=a.transAxes)
    # turn grid on
    a.grid(True,which='both',axis='both',linestyle='--',linewidth=0.5)

for a in ax[1,:]:
    a.set_xlabel('$V_g/T$')

# Adjust layout to remove gaps between subplots
plt.subplots_adjust(hspace=0.05,wspace=0.25)
plt.savefig('fig2.pdf',bbox_inches='tight')

plt.show()


follows red line in fig1(d)

Pauli, equivalent for seq. tunneling

dased = TUR bound

shaded areas = TUR forbidden

# TUR bounds
$2 \leq \frac{S}{I}\frac{V}{T_c}\frac{n_c-n}{n}$

for noise:
$ S \geq 2 I \frac{T_c}{V} \frac{n}{n_c-n}$

for current:

$I \leq \frac{S}{2}\frac{V}{T_c}\frac{n_c-n}{n}$ if $I>0$

$I \geq \frac{S}{2}\frac{V}{T_c}\frac{n_c-n}{n}$ if $I<0$

for efficiency:
$ n \leq \frac{n_c}{\frac{2IT_c}{SV}+1}$